<center><img src="https://www.mindinventory.com/blog/wp-content/uploads/2019/04/python-development-1200x500.png" width="1000"></center>

# Programa de Especialización en Python

### Manuel Sigüeñas, M.Sc.(c)
### Prof. Lenguajes de Programación para Ciencia de Datos / Agile Data Scientists / SCRUMStudy Certified Trainer
[msiguenas@socialdata-peru.com](msiguenas@socialdata-peru.com)

# Preparación de los datos de revisión de películas de IMDb para el procesamiento de texto

## Obtención del conjunto de datos de revisión de películas de IMDb

In [1]:
#pip install PyPrind

## Preprocesamiento del dataset de película en un formato más conveniente

In [1]:
import pyprind ### paquete para contabilizar el proceso de esperar 
import pandas as pd
import os

# cambiar la 'basepath' al directorio de la
# Conjunto de datos de película descomprimido

basepath = 'D:/Python/3. Nivel III/7-8/aclImdb_v1.tar/aclImdb'
## En el archivo se encuentra separados por carpetas, los comentarios de 50000 personas en archivo txt. 
## Tienes 2 carpetas que se dividen en train y en test, cada una de ellas tiene dos sub carpetas, pos y neg

labels = {'pos': 1, 'neg': 0} ## asignacion de una variables, si los comentarios estan en la carpeta "pos" es uno, sino es 0 
pbar = pyprind.ProgBar(50000) ## Número de archivos a cargar.
df = pd.DataFrame() ## Asignamos la función DataFrame a un objeto. 
for s in ('test', 'train'): ## Creamos un bucle, cuando s puede ser "test" o " train" 
    for l in ('pos', 'neg'): ## creamos un bucle dentro de otro, cuando l es "pos" o "neg" 
        path = os.path.join(basepath, s, l) ## asignamos al objeto "path" las rutas que pueden tomar, en total son 4. 
        for file in os.listdir(path): ## asignamos otra variable "file" que tendra la lista de todos los archivos text de cada ruta
            with open(os.path.join(path, file), ## abrimos los 50000 archivos
                      'r', encoding='utf-8') as infile: ## asignamos que el tipo de texto es "uft-8"
                txt = infile.read() #creamos un objeto donde se alamcenara todo lo extraido
            df = df.append([[txt, labels[l]]],  ## creamos un data frame donde asignaremos dos variables, los comentarios extraidos, y de donde proviene, si es negativo o positivo
                           ignore_index=True)
            pbar.update() 
df.columns = ['review', 'sentiment'] ## asignamos el nombre de las dos nuevas columnas

0% [##############                ] 100% | ETA: 00:01:46

Barajeando el DataFrame:

In [2]:
import numpy as np ## cambiamos el orden
## establecemos una semilla
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index)) ## barajeamos el data frame

Opcional: Guardar los datos ensamblados como archivo CSV:

In [3]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8') ## asignamos un nombre al nuevo data frame

In [4]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8') ## abrimos el archivo y lo guardamos en el objeto df
df.head(3) ## observamos los 3 primeros textos

,review,sentiment
0,I thought this movie was absolutely hilarious....,1
1,Pluses: Mary Boland is delightfully on edge as...,0
2,This movie is not very bad tjough. But one can...,0


<br>
<br>

# Modelo de < bolsa de palabras \>

...

## Transformando documentos en vectores de características

Al llamar al método fit_transform en CountVectorizer, acabamos de construir el vocabulario del modelo de bolsa de palabras y transformamos las tres frases siguientes en vectores de características dispersos:
<br>

1. El sol brilla
2. El clima es dulce
3. El sol está brillando, el clima es dulce, y uno y uno es dos

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer ## llamar a la función  "feature_extraction.text" para vectorizar(separar) las palabras 

count = CountVectorizer() #asignamos la función a un objeto 
docs = np.array([ ## creamos una lista 
        'El sol brilla',
        'El clima es dulce',
        'El sol está brillando, el clima es dulce, y uno y uno es dos'])
bag = count.fit_transform(docs) ## vectorizamos y contamos las palabras

Ahora vamos a imprimir el contenido del vocabulario para obtener una mejor comprensión de los conceptos subyacentes:

In [6]:
print(count.vocabulary_)

{'el': 5, 'sol': 8, 'brilla': 0, 'clima': 2, 'es': 6, 'dulce': 4, 'está': 7, 'brillando': 1, 'uno': 9, 'dos': 3}


Como podemos ver al ejecutar el comando anterior, el vocabulario se almacena en un diccionario de Python, que asigna las palabras únicas que se asignan a índices enteros. A continuación vamos a imprimir los vectores de características que acabamos de crear:

Cada posición de índice en los vectores de entidad que se muestran aquí corresponde a los valores enteros que se almacenan como elementos de diccionario en el vocabulario CountVectorizer. Por ejemplo, la primera entidad en la posición de índice 0 se asemeja al recuento de la palabra y, que solo se produce en el último documento, y la palabra está en la posición de índice 1 (la 2a entidad de los vectores de documento) se produce en las tres oraciones. Esos valores en los vectores de entidad también se denominan frecuencias de términos sin procesar: *tf (t,d)*—el número de veces que se produce un término t en un documento *d*.

In [7]:
print(bag.toarray())

[[1 0 0 0 0 1 0 0 1 0]
 [0 0 1 0 1 1 1 0 0 0]
 [0 1 1 1 1 2 2 1 1 2]]


<br>

## Evaluación de la relevancia de la palabra a través de la frecuencia de los documentos inversos en frecuencia de términos

In [8]:
np.set_printoptions(precision=2)

Cuando analizamos datos de texto, a menudo encontramos palabras que se producen en varios documentos de ambas clases. Las palabras que ocurren con frecuencia normalmente no contienen información útil o discriminatoria. En esta subsección, aprenderemos acerca de una técnica útil llamada frecuencia de documentos inversa de frecuencia de término (tf-idf) que se puede utilizar para reducir el peso de las palabras que ocurren con frecuencia en los vectores de entidades. El tf-idf se puede den como el producto del término frecuencia y la frecuencia inversa del documento:

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

Aquí el tf(t, d) es el término frecuencia que introdujimos en la sección anterior,
y la frecuencia inversa del documento *idf(t, d)* se puede calcular como: 

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

donde $n_d$ es el número total de documentos, y *df(d, t)* es el número de documentos *d* que contienen el término *t*. Tenga en cuenta que agregar la constante 1 al denominador es opcional y sirve para asignar un valor distinto de cero a los términos que se producen en todas las muestras de entrenamiento; el registro se utiliza para asegurarse de que las frecuencias de documento bajas no reciben demasiado peso.

Scikit-learn implementa otro transformador, el 'TfidfTransformer', que toma el término sin procesar frecuencias de 'CountVectorizer' como entrada y las transforma en tf-idfs:

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.72 0.   0.   0.   0.   0.43 0.   0.   0.55 0.  ]
 [0.   0.   0.53 0.   0.53 0.41 0.53 0.   0.   0.  ]
 [0.   0.28 0.22 0.28 0.22 0.33 0.43 0.28 0.22 0.57]]


Como vimos en la subsección anterior, la palabra tiene la mayor frecuencia de términos en el 3er documento, siendo la palabra que ocurre con más frecuencia. Sin embargo, después de transformar el mismo vector de características en tf-idfs, vemos que la palabra es
ahora asociado con un tf-idf relativamente pequeño (0,45) en el documento 3, ya que es
documentos 1 y 2 y, por lo tanto, es poco probable que contenga información útil y discriminatoria.


Sin embargo, si calcularamos manualmente los tf-idfs de los términos individuales en nuestros vectores de características, habríamos notado que el 'TfidfTransformer' calcula el tf-idfs ligeramente diferente en comparación con las ecuaciones de libros de texto estándar que demos anteriormente. Las ecuaciones para el idf y el tf-idf que se implementaron en scikit-learn son:

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

La ecuación tf-idf que se implementó en scikit-learn es la siguiente:

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

Aunque también es más típico normalizar las frecuencias de términos sin procesar antes de calcular el tf-idfs, el 'TfidfTransformer' normaliza directamente el tf-idfs.

De forma predeterminada ('norm''l2''), TfidfTransformer de scikit-learn aplica la normalización L2, que devuelve un vector de longitud 1 dividiendo un vector de entidades no normalizado *v* por su norma L2:

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$

Para asegurarnos de que entendemos cómo funciona TfidfTransformer, caminemos
a través de un ejemplo y calcular el tf-idf de la palabra está en el 3er documento.

La palabra tiene una frecuencia de término de 3 (tf a 3) en el documento 3, y la frecuencia del documento de este término es 3 ya que el término se produce en los tres documentos (df 3). Por lo tanto, podemos calcular el idf de la siguiente manera:

$$\text{idf}("is", d3) = log \frac{1+3}{1+3} = 0$$

Ahora, para calcular el tf-idf, simplemente necesitamos agregar 1 a la frecuencia inversa del documento y multiplicarlo por el término frecuencia:

$$\text{tf-idf}("is",d3)= 3 \times (0+1) = 3$$

In [10]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


Si repetimos estos cálculos para todos los términos del 3er documento, obtendríamos los siguientes vectores tf-idf: [3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]. Sin embargo, observamos que los valores de este vector de entidad son diferentes de los valores que obtuvimos del TfidfTransformer que usamos anteriormente. El paso nal que nos falta en este cálculo tf-idf es la l2-normalización, que se puede aplicar de la siguiente manera:

$$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2, 3.0^2, 3.39^2, 1.29^2, 1.29^2, 1.29^2, 2.0^2 , 1.69^2, 1.29^2]}}$$

$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$

$$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$$

Como podemos ver, los resultados coinciden con los resultados devueltos por scikit-learn 'TfidfTransformer' (abajo). Puesto que ahora entendemos cómo se calculan los tf-idfs, procedamos a las siguientes secciones y apliquemos esos conceptos al conjunto de datos de revisión de películas.

In [11]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf 

array([0.  , 1.69, 1.29, 1.69, 1.29, 2.  , 2.58, 1.69, 1.29, 3.39])

In [12]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0.  , 0.28, 0.22, 0.28, 0.22, 0.33, 0.43, 0.28, 0.22, 0.57])

<br>

## Limpieza de datos de texto

In [13]:
df.loc[0, 'review'][-50:] ## Localizamos un cometario del texto

"'d you expect. <br /><br />Overall,just hilarious."

In [14]:
import re ## como se puede observar en el texto de arriba esta aun con parte del lenguaje html
def preprocessor(text): ## definimos una función llamada preprocessor, que requerira al argumento "text"
    text = re.sub('<[^>]*>', '', text)  ## el objeto llamado text se modificara para que elimine el código del lenguaje html
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', ## asignamos un nuevo ojeto que almacenara a los emoticones del archivo modificado
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) + 
            ' '.join(emoticons).replace('-', ''))## el archivo text es modificado 
    return ## text es el obejto a retornar es los modificado

In [15]:
preprocessor(df.loc[0, 'review'][-50:]) ### probamos con una parte de la data 

In [16]:
preprocessor("</a>This :) is :( a test :-)!") ## probamos la retención de los emoticones

In [17]:
df['review'] = df['review'].apply(preprocessor) ## aplicamos la función creada a la data entera

<br>

## Procesamiento de documentos en tokens

In [18]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):  ## función para tpkenizar los datos
    return text.split()


def tokenizer_porter(text): ## función para tokenizar los datos y llevarlos a su raiz
    return [porter.stem(word) for word in text.split()]

In [19]:
tokenizer('a los corredores les gusta correr y por lo tanto corren')

['a',
 'los',
 'corredores',
 'les',
 'gusta',
 'correr',
 'y',
 'por',
 'lo',
 'tanto',
 'corren']

In [20]:
tokenizer_porter('a los corredores les gusta correr y por lo tanto corren')

['a',
 'lo',
 'corredor',
 'le',
 'gusta',
 'correr',
 'y',
 'por',
 'lo',
 'tanto',
 'corren']

In [21]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\SOCIAL
[nltk_data]     DATA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
from nltk.corpus import stopwords
### probamos el "tokenizer_porter"
stop = stopwords.words('Spanish')
[w for w in tokenizer_porter('a un corredor le gusta correr y corre mucho')[-10:]
if w not in stop]

['corredor', 'gusta', 'correr', 'corr']

<br>
<br>

<br>
<br>

## Modelado de temas

### Descomponer documentos de texto con LDA

### Latent Dirichlet Allocation con scikit-learn







In [23]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,I thought this movie was absolutely hilarious....,1
1,Pluses: Mary Boland is delightfully on edge as...,0
2,This movie is not very bad tjough. But one can...,0


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [26]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10, ## número de temas a conseguir, antes atributo n_topics
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [27]:
lda.components_.shape

(10, 5000)

In [28]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
script worst action poor budget
Topic 2:
music dvd version original song
Topic 3:
guy stupid kids girl minutes
Topic 4:
war woman wife men death
Topic 5:
horror gore effects monster blood
Topic 6:
documentary true feel american family
Topic 7:
game series episode episodes cartoon
Topic 8:
role performance play plays actress
Topic 9:
action john plays role guy
Topic 10:
book comedy series read humor


Basándonos en la lectura de las 5 palabras más importantes para cada tema, podemos adivinar que el LDA identificó los siguientes temas:
    
1. Por lo general, películas malas (no es realmente una categoría de tema)
2. Películas sobre familias
3. Películas de guerra
4. Películas de arte
5. Películas de crimen
6. Películas de terror
7. Comedias
8. Películas de alguna manera relacionadas con programas de televisión
9. Películas basadas en libros
10. Películas de acción

Para confirmar que las categorías tienen sentido en función de las reseñas, vamos a trazar 5 películas de la categoría de películas de terror (categoría 6 en la posición de índice 5):

In [29]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


Horror movie #1:
"Shadows" is often acclaimed as the film that was the breakthrough for American independent cinema. Whether thats true or not, it is an undeniably important film, one whose influence can be traced all the way to today's Sundance fodder. Here is a film which tackles controversial topics of the day (n ...

Horror movie #2:
I won't go to a generalization, and say it's the best love story of all time, as some have said. That's fine, people feel very deeply about this film, you either love it I believe...or you simply hate it. I don't want to say, the best of all,because that is simply too 'broad' for me to make a statem ...

Horror movie #3:
In 2004, I wrote the following statements on an IMDb message board when a user wondered if The Best Years of Our Lives was a forgotten movie: <br /><br />***** To me watching this movie is like opening up a time capsule. I think in many ways "The Best Years of Our Lives" is probably one of the more  ...


Usando el ejemplo de código anterior, imprimimos los primeros 300 palabras de las 3 comentarios sobre películas de terror y, de hecho, podemos ver que las críticas - aunque no sabemos a qué película exacta pertenecen - suenan como reseñas de películas de terror, de hecho. (Sin embargo, se podría argumentar que las #2 de películas también podrían pertenecer a la categoría de tema 1.)